In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv('train_data.csv', index_col=0)
test_kaggle = pd.read_csv('test_kaggle.csv', index_col=0)

In [4]:
train_data.columns

Index(['person', 'label', 'Bom', 'Bom - Sem Touch ID', 'Excelente',
       'Muito Bom', 'Novo', 'visitas_dom', 'visitas_lun', 'visitas_mar',
       'visitas_mier', 'visitas_jue', 'visitas_vier', 'visitas_sab',
       'conversiones_mean', 'vio_mas_de_5_veces', 'vio_color_mas_vendido',
       'returns_por_persona_mean', 'solicito_brand_listing',
       'productos_vistos_mean', 'busca_palabra_popular', '128GB', '16GB',
       '256GB', '32GB', '4GB', '512MB', '64GB', '8GB', 'periodo_y',
       'ad campaign hit_mean_y', 'brand listing_mean_y', 'checkout_mean_y',
       'conversion_mean_y', 'generic listing_mean_y', 'lead_mean_y',
       'search engine hit_mean_y', 'searched products_mean_y',
       'staticpage_mean_y', 'viewed product_mean_y', 'visited site_mean_y',
       'tiempo_ultima_conversion', 'timestamp_x', 'tiempo_ultimo_checkout',
       'timestamp_y', 'tiempo_ultima_visita', 'timestamp_x.1',
       'tiempo_ultimo_lead', 'tiempo_entre_conversiones_mean_log_x',
       'timestamp_y.

In [5]:
features = ['Bom', 'Bom - Sem Touch ID', 'Excelente',
       'Muito Bom', 'Novo', 'visitas_dom', 'visitas_lun', 'visitas_jue', 'visitas_vier',
       'conversiones_mean', 'vio_mas_de_5_veces', 'vio_color_mas_vendido',
       'returns_por_persona_mean', 'solicito_brand_listing',
       'productos_vistos_mean', '128GB',
       '16GB', '256GB', '32GB', '4GB', '512MB', '64GB', '8GB', 'periodo_y',
       'ad campaign hit_mean_y', 'brand listing_mean_y', 'checkout_mean_y',
       'conversion_mean_y', 'generic listing_mean_y', 'lead_mean_y',
       'search engine hit_mean_y', 'searched products_mean_y',
       'staticpage_mean_y', 'viewed product_mean_y', 'visited site_mean_y',
       'tiempo_ultima_conversion','tiempo_ultimo_checkout','tiempo_ultima_visita','tiempo_ultimo_lead','vio_producto_popular','Samsung', 'tiempo_entre_conversion_checkout', 'checkout',
       'conversion', 'proporcion','cant_conversiones_ultimo_periodo','dias_desde_ultimo_evento']

In [6]:
randomForest_KFold = RandomForestClassifier()

X = np.array(train_data[features])
y = np.array(train_data['label'])

X_train,X_test,Y_train,Y_test = train_test_split(X,y)

In [7]:
kfold = KFold(n_splits=10)

In [8]:
mejor_score = 0
for train, test in kfold.split(X_train, Y_train):
    train_datas = train_data.iloc[train]
    test_datas = train_data.iloc[test]
    
    X_train_data = train_datas[features]
    Y_train_data = train_datas['label']
    
    X_test_data = test_datas[features]
    Y_test_data = test_datas['label']
    
    randomForest_KFold.fit(X=X_train_data,y=Y_train_data)
    print(roc_auc_score(Y_test_data,randomForest_KFold.predict(X_test_data)))
    if (roc_auc_score(Y_test_data, randomForest_KFold.predict(X_test_data))>mejor_score):
        mejor_score = roc_auc_score(Y_test_data,randomForest_KFold.predict(X_test_data))
        X_train_optimo = X_train_data
        Y_train_optimo = Y_train_data
        X_test_optimo = X_test_data
        Y_test_optimo = Y_test_data

0.498551774076756
0.5049588776003869
0.5182386739207809
0.5242732558139535
0.5117086193745234
0.5212125696808247
0.504042715484363
0.5098002244668911
0.5398603082851637
0.5468047527296083


In [9]:
randomForest_KFold.get_params().keys()

dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [10]:
params={'n_estimators':list(range(10,50)),'max_features':list(range(1,20))}

In [11]:
cv = GridSearchCV(estimator=randomForest_KFold,param_grid=params,scoring='roc_auc',cv=5)
cv.fit(X_train_optimo,Y_train_optimo)


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], 'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [12]:
cv.best_score_

0.8334595984190729

In [13]:
randomForest = cv.best_estimator_

In [ ]:
randomFores

In [15]:
randomForest.fit(X_train_optimo,Y_train_optimo)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=45, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
predict = randomForest.predict_proba(X_test_optimo)

In [20]:
predict

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [0.97777778, 0.02222222],
       [1.        , 0.        ],
       [1.        , 0.        ]])

# Gradient Boosting Classifier

In [85]:
#GRADIENT BOOSTING = 83%

In [72]:
gbc = GradientBoostingRegressor()

In [73]:
gbc.fit(X_train,Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [74]:
gbc.score(X_test,Y_test)

0.13904431505377757

In [75]:
gbc.score(X_train,Y_train)

0.1858865260336722

In [76]:
test_kaggle['prediction'] = gbc.predict(test_kaggle[features])

In [77]:
test_kaggle['prediction'] = test_kaggle['prediction'].transform(lambda x: 0.0 if x < 0.0 else x)

# Test Kaggle

In [84]:
#RANDOM FOREST = 72%

In [9]:
test_kaggle = randomForest.predict_proba(test_kaggle[features])

In [81]:
test_kaggle['prediction'] = test_kaggle['prediction'].transform(lambda x: 0.0 if x < 0.0 else x)

In [82]:
submit_kaggle = pd.DataFrame({'person':test_kaggle['person'],'label':test_kaggle['prediction']})
submit_kaggle.reset_index(inplace = True)
submit_kaggle.drop(columns='index',inplace=True)
submit_kaggle.set_index(keys='person',inplace=True)

In [83]:
submit_kaggle.to_csv('submit_kaggle.csv')